<!-- ## SMALL-SCALE CROP-MAPPING(Food security) -->

<center><h1>Carbon Hawk</h1> </center>

In [10]:
#Packages used:
#Uncomment and install the packages below via your terminal
!pip install geopandas
!pip install geemap
!pip install pip install rw-dynamicworld-cd==0.0.1

#Importation of variAous Libraries or packages...
import os
from os import path as op
import geopandas as gdp
import geemap
import ee


# Making module or packages collection attribute became callable..
import collections
collections.Callable = collections.abc.Callable

#plotting packages
import pandas as pd#Used for data analysis also include DataFrame df data structure
import matplotlib.pyplot as plt#For plotting of dataframes
import numpy as np
# import altair as alt#declarative visualization(Used for charting)

import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl
from ipywidgets import HTML
from IPython.display import display, clear_output
from ipywidgets import AppLayout

from geemap import chart #for chart plotting


#packages used in database connection.
import pandas.io.sql as sqlio
import psycopg2 as ps

#Main function is to out various warning that might arise such as new upadates in package version
import warnings
warnings.filterwarnings("ignore")

#Download wri_change_detection from https://pypi.org/project/rw-dynamicworld-cd/0.0.1/ 
# !pip install pip install rw-dynamicworld-cd==0.0.1
from wri_change_detection import preprocessing as npv
from wri_change_detection import gee_classifier as gclass
from wri_change_detection import post_classification_filters as pcf

Map=geemap.Map()
from geemap import Map
from ipywidgets import Button, Layout

map_layout= Map(layout=Layout(height='100px'))
map_layout.setCenter(-81.056, 25.4157
,10)
 
map_layout.add_basemap(basemap='HYBRID')

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
#Setting various buttons for the final dsplay of the results and the output widgets or setting the intractive keys
style = {'description_width': 'initial'}
#General output widget
output_widget = widgets.Output(layout={'border': '1px solid red','align-items':'center','background-color':'red'})

#Widget used to display crop health
output_widget_health = widgets.Output(layout={'border': '1px solid black','marginright':'150px'})

#General output control tool.
output_control = WidgetControl(widget=output_widget, position='topright')
#Map output control
map_layout.add_control(output_control)

#BUTTONS  and TEXT SECTION 
#styling method that can be used-----'primary', 'success', 'info', 'warning', 'danger',)

#AOI Area of Interest Selection of study area.
AOI = widgets.Button(
    description='Drawn Study Area',
    button_style='success',
    tooltip='Draw you boundaries',
    style=style
)


Image_Preprocessing = widgets.Button(
    description='Image Preprocessing',
    button_style='info',
    tooltip='Image Preprocessing process',
    style=style
)
Update_Mask = widgets.Button(
    description='Mask Mangrove',
    button_style='danger',
    tooltip='Mask Cropland',
    style=style
)

load_model = widgets.Button(
    description='Classification & Training points',
    button_style='primary',
    tooltip='Random sampling of points',
    style=style
)

Export_data = widgets.Button(
    description='Calculate Mean',
    button_style='success',
    tooltip='Generate Mean Value',
    style=style
)
GPP_EXECUTE = widgets.Button(
    description='Map Mangrove Forest',
    button_style='warning',
    tooltip='Gross primary product calculatation',
    style=style
)

Reset_button= widgets.Button(
    description='Reset',
    button_style='danger',
    tooltip='reset by use of this button',
    style=style
)
Data_Input = widgets.Button(
    description='Predict Deforestation',
    button_style='info',
    tooltip='Predict Deforestation',
    style=style
)

ND_execute = widgets.Button(
    description='Forest Analysis',
    button_style='primary',
    tooltip='Forest Analysis',
    style=style
)

LULCC = widgets.Button(
    description='Dynamic LULC',
    button_style='info',
    tooltip='Dynamic Worlds',
    style=style
)


In [ ]:
output_widget88 = widgets.Output(layout={'border': '1px solid black'})
output_control88 = WidgetControl(widget=output_widget88, position='bottomright')
map_layout.add_control(output_control88)

In [ ]:
style = {'description_width': 'initial'}
uploader = widgets.FileUpload(
    description='Upload data',
    accept='.zip, .json, .geojson',
    multiple=False,
    button_style='primary',
    style=style,
)

submit = widgets.Button(
    description='Display data', button_style='success', tooltip='Click me', style=style
)

reset = widgets.Button(
    description='Reset', button_style='warning', tooltip='Click me', style=style
)

In [ ]:
with output_widget88:
    print('Upload shapefile or \ngeojson as a zip file')
    display(uploader)
    display(submit)
    display(reset)

In [ ]:
def get_vector(upload_widget, out_dir=None):

    import zipfile
    import glob

    if out_dir is None:
        out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    vector = None

    try:
        [uploaded_file] = upload_widget.value
        file = upload_widget.value[uploaded_file]
        name = file['metadata']['name']
        content = file['content']
        out_file = os.path.join(out_dir, name)
        with open(out_file, "wb") as fp:
            fp.write(content)

        if name.endswith('.zip'):
            with zipfile.ZipFile(out_file, "r") as zip_ref:
                extract_dir = os.path.join(
                    out_dir, name[:-4] + "_" + geemap.random_string(3)
                )
                zip_ref.extractall(extract_dir)
                files = glob.glob(extract_dir + '/*.shp')
                if len(files) > 0:
                    shp = files[0]
                    vector = geemap.shp_to_ee(shp)
                else:
                    files = glob.glob(extract_dir + '/*.geojson')
                    if len(files) > 0:
                        geojson = files[0]
                        vector = geemap.geojson_to_ee(geojson)
        else:
            vector = geemap.geojson_to_ee(out_file)

    except Exception as e:
        print(e)

    return vector

In [ ]:
def submit_clicked(b):
    if uploader._counter > 0:
        map_layout.default_style = {'cursor': 'wait'}
        try:
            fc = get_vector(uploader)
            layer_name = 'Layer ' + geemap.random_string(3)
            map_layout.addLayer(fc, {}, layer_name)
            map_layout.centerObject(fc)
            uploader.value.clear()
            uploader._counter = 0
        except Exception as e:
            print(e)
        map_layout.default_style = {'cursor': 'pointer'}


submit.on_click(submit_clicked)


def reset_clicked(b):

    map_layout.layers = map_layout.layers[:3]
    output_widget88.clear_output()
    with output_widget88:
        print('Upload shapefile or \ngeojson as a zip file')
        display(uploader)
        display(submit)
        display(reset)
    uploader.value.clear()
    uploader._counter = 0


reset.on_click(reset_clicked)

In [ ]:
#User instructions manual.
instructions=widgets.Dropdown(
    options=['Please follow this steps sequentially',
             '1.Drawn study area or upload',
             '2.Carry out image pre-processing',
             '3.Map Mangrove',
             '4.Mask out Mangrove',
             '5.Image classification',
             '6.Alternatively use Dynamic World for image classification(if you want to work with another study area)',
             '7.Mangrove Analysis and prediction',
             '8.Reset Map'],
    value='Please follow this steps sequentially',
    description='Instruction:',
    disabled=False,
)


In [ ]:
#Arrange the layout
from ipywidgets import AppLayout
verticalBox = output_widget
vBox1 = widgets.VBox([map_layout])


vBox2 = widgets.VBox([instructions,AOI, Image_Preprocessing,GPP_EXECUTE,Update_Mask,load_model,LULCC,Data_Input,Reset_button])

AppLayout(header=None,
          left_sidebar=vBox1,
          right_sidebar=vBox2,
          footer=None,
          step= 1,
          pane_widths=[3, 3, 1],
          pane_heights=[1, 1, '100%'])

In [ ]:
#Trial to avoid printing error in the code but a message to the user
def AOI_clicked(b):
        with output_widget:
#             output_widget.clear_output()
           
            try:
                global boundaries
                
                boundaries=map_layout.user_rois
                
#                 boundaries=ee.FeatureCollection("projects/ee-mosongjnvscode/assets/Mangrove_region_assets")

            
                map_layout.addLayer(boundaries,{},"Region of Interest")
              
         
            except Exception as e:
                print('Please select Your area of Interest.....')
            else:
                print("Successfully drawn your Region of Interest ")
                output_widget.clear_output()
        
AOI.on_click(AOI_clicked)

In [ ]:
#Trial to avoid printing error in the code but a message to the user
def AOI_clicked(b):
        with output_widget:
            map_layout.remove_legends() 
            map_layout.remove_colorbars()
            
            global npp
            npp=ee.ImageCollection("MODIS/006/MOD17A3H")
            global gpp
            gpp=ee.ImageCollection("MODIS/006/MYD17A2H")
            map_layout.addLayer(boundaries,{},"boundaries")
#             map_layout.centerObject(boundaries,17)
            
            
            # Retrive dataset as a collection of images from Earth Engine Catalog
            global mangrove_images_landsat
            mangrove_images_landsat = ee.ImageCollection('LANDSAT/MANGROVE_FORESTS')
            
            mangrove_images_landsat = mangrove_images_landsat.first().clip(boundaries )
            
            mangrovesVis = {
                min: 0,
                max: 1.0,
                'palette': ['22f302'],
            }
            
            map_layout.addLayer(mangrove_images_landsat, mangrovesVis, 'Mangroves')
            
GPP_EXECUTE.on_click(AOI_clicked)

In [ ]:
# Click event handler for Image processing process e.g landsat data

def preprocess_img_clicked(b):
     with output_widget:
            output_widget.clear_output()
            
            # select year
            year = 2016
            
            global landsat_image
            landsat_image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
            .filterBounds(boundaries) \
            .filterDate(f'{year}-01-01', f'{year}-12-31') \
            .select('B[1-7]') \
            .sort('CLOUD_COVER') \
            .first()\
            .clip(boundaries)
            
            vis_params = {
                'min': 0,
                'max': 3000,
                'bands': ['B4', 'B3', 'B2']
            }
#             map_layout.centerObject(boundaries, 12)
            map_layout.addLayer(boundaries,{},"boundaries")
            map_layout.addLayer(landsat_image, vis_params, "Landsat-8")
        
Image_Preprocessing.on_click(preprocess_img_clicked)


In [ ]:
#Masking mangrove forest
def AOI_clicked(b):
    with output_widget:
        output_widget.clear_output()
        global mangrove_mask
        mangrove_mask = landsat_image.updateMask(mangrove_images_landsat.eq(1))
        global non_mangrove_mask
        non_mangrove_mask = landsat_image.updateMask(mangrove_mask.unmask().Not())
        map_layout.addLayer(mangrove_mask, {'min': 0, 'max': 10000, 'bands': ['B1']}, 'mangrove mask', True)
    
Update_Mask.on_click(AOI_clicked)

In [ ]:
#Masking mangrove forest
def load_model_clicked(b):
    
    with output_widget:
        output_widget.clear_output()
        
# mangrove_training_pts
        
        mangrove_training_pts = mangrove_mask.sample(**{
    'region': mangrove_mask.geometry(),
    'scale': 30,
    'numPixels': 10000,
    'seed': 5343,
    'geometries': True})
# non_mangrove_training_pts 
        non_mangrove_training_pts = non_mangrove_mask.sample(**{
    'region': non_mangrove_mask.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True})
        vis_params = {
    'min': 0,
    'max': 100,
    'bands': ['B4']}
        
        mangrove_color = 'eb0000'
        non_mangrove_color = '1c5f2c'
        
#         map_layout.addLayer(mangrove_mask, {'min': 0, 'max': 10000, 'bands': ['B1']}, 'mangrove mask', True)
        map_layout.addLayer(mangrove_training_pts, {'color': mangrove_color}, 'Mangrove Sample')
        map_layout.addLayer(non_mangrove_mask, {}, 'non mangrove mask', True)
        map_layout.addLayer(non_mangrove_training_pts, {'color': non_mangrove_color}, 'non mangrove training', True)
load_model.on_click(load_model_clicked)

In [ ]:
#Trial to avoid printing error in the code but a message to the user
def AOI_clicked(b):
        with output_widget:
            output_widget.clear_output()
            forestChange = ee.Image("UMD/hansen/global_forest_change_2019_v1_7")
            
            projection_ee = forestChange.projection()
            projection = projection_ee.getInfo()
            crs = projection.get('crs')
            crsTransform = projection.get('transform')
            scale = projection_ee.nominalScale().getInfo()
            
            countryBoundaries = ee.FeatureCollection('projects/resource-watch-gee/gadm36_0')
            DRCBoundary = countryBoundaries.filterMetadata('GID_0','equals','COD').first().geometry()
            
            primaryForest = ee.Image('UMD/GLAD/PRIMARY_HUMID_TROPICAL_FORESTS/v1/2001')

# select tree cover loss band and unmask with value 0 to represent no loss
            tclYear = forestChange.select(['lossyear']).unmask(0)

# define mask of primary forest areas
            primaryForestValid = primaryForest.eq(1);
# define mask for where Hansen data is valid
            forestChangeValid = forestChange.select('datamask').eq(1)
# define mask of tree cover loss > 30%
            forestCoverValid = forestChange.select('treecover2000').gte(30)

# create final mask where (primaryForestValid = 1) AND (forestChangeValid = 1) AND (forestCoverValid = 1)
            treeCoverLossValid = primaryForestValid.bitwiseAnd(forestCoverValid).bitwiseAnd(forestChangeValid)

# update tree cover loss year to mask invalid areas
            tclYearMasked = tclYear.updateMask(treeCoverLossValid)
# now tclYearMasked = {0 when tree cover loss did not occur, 1-19 for year when tree cover loss occured} only in valid areas
            tclPalette = ['#0051ff',
            '#f7f4f9','#f7f4f9',
            '#e7e1ef','#e7e1ef',
            '#d4b9da','#d4b9da',
            '#c994c7','#c994c7',
            '#df65b0','#df65b0',
            '#e7298a','#e7298a',
            '#ce1256','#ce1256',
            '#980043','#980043',
            '#67001f','#67001f']
            tclViz = {'min': 0, 'max': 19, 'palette': tclPalette}
            map_layout.addLayer(forestChangeValid.updateMask(forestChangeValid).clip(boundaries),
              {'min': 0, 'max': 1, 'palette': ['#f9261b','#5a1bf9']},name='Hansen Valid')
            map_layout.addLayer(forestCoverValid.updateMask(forestCoverValid).clip(boundaries),
              {'min': 0, 'max': 1, 'palette': ['#f7f91b','#28ce4c']},name='Tree Cover > 30')
            map_layout.addLayer(primaryForestValid.updateMask(primaryForestValid).clip(boundaries),
              {'min': 0, 'max': 1, 'palette': ['#f9801b','#9d1bf9']},name='Primary Forest')
            map_layout.addLayer(forestChange.select(['lossyear']).clip(boundaries),tclViz,name='Tree Cover Loss UnMasked')
            map_layout.addLayer(tclYearMasked.clip(boundaries),tclViz,name='Tree Cover Loss Masked')
            
            tclEarlyLoss = tclYearMasked.expression(
    '(year>0 && year<12)', {
      'year': tclYearMasked.select('lossyear')})
# Define mask when tree cover occurred after 2017
            tclLaterLoss = tclYearMasked.expression(
    '(year>17)', {
      'year': tclYearMasked.select('lossyear')
})

# Define binary variable for tree cover loss that occurred from 2012 through 2017,
# loss that occurred after 2017 is marked as 0
            tclReferenceLoss = tclYearMasked.expression(
    '(year>11 && year<18) ? 1 : 0', {
      'year': tclYearMasked.select('lossyear')})

# Define mask for tree cover loss that occurred from 2012 through 2017
            referenceTreeCoverLossValid = treeCoverLossValid.bitwiseAnd(tclEarlyLoss.eq(0))

# Mask tree cover loss years to get binary 0 for no tree cover loss and 1 for 
            tclReferenceLoss = tclReferenceLoss.updateMask(referenceTreeCoverLossValid).gt(0)
            tclReferenceLoss = tclReferenceLoss.rename('loss')

# Map layers to double check!
            map_layout.addLayer(tclYearMasked,tclViz,name='Tree Cover Loss Masked')
            map_layout.addLayer(tclEarlyLoss,{'min': 0, 'max': 1, 'palette': ['#f9261b','#5a1bf9']},name='Early Loss')
            map_layout.addLayer(tclLaterLoss.clip(boundaries),{'min': 0, 'max': 1, 'palette': ['#f9261b','#5a1bf9']},name='Later Loss')
            map_layout.addLayer(tclReferenceLoss,{'min': 0, 'max': 1, 'palette': ['#f9261b','#5a1bf9']},name='Historical Loss')   
ND_execute.on_click(AOI_clicked)

In [ ]:
#Creation of an output widgets to ouput outside the map.
user_out = widgets.Output(layout={'border': '1px solid green'})
user_out

In [ ]:
# Click event handler for Image processing process e.g sentinel data

def submit_clicked(b):
     with output_widget:
            try:
                output_widget.clear_output()
                map_layout.remove_legends()
                start_date="2018-01-01"#Set start_date(yy/mon/day)
                end_date="2022-03-31"#Set End_date(yy/mon/day)
            
                global region # Set the region of interest by simply drawing a polygon on the map
                region = boundaries
                if region is None:
                    region = ee.Geometry.BBox(-61.18116254022864,6.614609265030635, -61.18116254022864,6.769074870340775)

                image = geemap.dynamic_world_s2(region, start_date, end_date)
                vis_params3 = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
                global landcover
                # Create Dynamic World land cover composite
                landcover = geemap.dynamic_world(region, start_date, end_date, return_type='hillshade')

                # Add legend to the map
                map_layout.addLayer(landcover.clip(boundaries),{}, 'LULCC') 
                map_layout.add_legend(title="Land Cover", builtin_legend='Dynamic_World')
                

            except Exception as e:
                print('Sorry an error occurred...please try again')
            else:
                print("LULC attained successfully")
        
LULCC.on_click(submit_clicked)

In [ ]:
def reset_me(b):
    with output_widget:
        output_widget.clear_output()
        try:
            map_layout.remove_legends() 
            map_layout.remove_colorbar()
            map_layout.remove_drawn_features()
            map_layout.remove_ee_layer('Dynamic_World')
            map_layout.remove_ee_layer("Mangroves")
            map_layout.remove_ee_layer("Region of Intrest")
            map_layout.remove_ee_layer('Landsat-8')
            map_layout.remove_ee_layer('mangrove mask')
            map_layout.remove_ee_layer('Mangrove Sample')
            map_layout.remove_ee_layer('non mangrove mask')
            map_layout.remove_ee_layer('non mangrove training')
            map_layout.remove_ee_layer('boundaries')
            
            
            
        except Exception as e:
            print("Sorry an error just occured")
        else:
            print("reset successfully....")

Reset_button.on_click(reset_me)